<a href="https://colab.research.google.com/github/emilyj19/TS/blob/master/SQF_RNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch

In [3]:
pip install gluonts

     |████████████████████████████████| 225kB 4.8MB/s 
     |████████████████████████████████| 28.4MB 1.2MB/s 
     |████████████████████████████████| 4.8MB 22.3MB/s 
     |████████████████████████████████| 194kB 42.4MB/s 
     |████████████████████████████████| 13.8MB 35.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
ERROR: spacy 2.1.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: fastai 1.0.53.post2 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: blis 0.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5

In [0]:
import numpy as np 
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt 
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
root_path = 'gdrive/My Drive/FinancialTS/JPmarket_dataset.npz' 
data = np.load(root_path)
data.files

['train_ratios', 'test_ratios', 'train_volumes', 'test_volumes']

In [0]:
train_ratios = data['train_ratios']
test_ratios = data['test_ratios']
train_vols = data['train_volumes']
test_vols = data['test_volumes']

In [0]:
mini_train_ratios = train_ratios[0:50]
mini_test_ratios = test_ratios[0:50]

In [0]:
#need to change this to create windows for all stocks 

def split_data(n_days, data): #data for one stock in the form [days, bins]
  no_ts = data.shape[0]-n_days
  length_ts = n_days*data.shape[1]
  new_data = np.zeros((no_ts,length_ts))
  for j in range(no_ts):
    for i in range(n_days): 
      new_data[j,64*i:64*i + 64] = data[j+i,:]
  
  return new_data

training_data = split_data(180, train_ratios[0])

In [0]:
def make_test_data(train_data, test_data, n_days, stock_index):
  all_data = np.concatenate((train_data, test_data), axis = 1)
  
  one_data = all_data[stock_index]
  
  #now want to take only sets of n_days which contain the test data too 
  no_ts = test_data.shape[1]
  length_ts = n_days*one_data.shape[1]
  new_data = np.zeros((no_ts, length_ts))
  
  for j in range(no_ts): 
    for i in range(n_days): 
      new_data[j, 64*i:64*i + 64] = one_data[train_data.shape[1]+j-180+i+1,:]
      
  return new_data


testing_ratios = make_test_data(train_ratios, test_ratios, 180, 0)

In [0]:
def getbd_from_theta(theta): 
  gamma, beta, delta = theta
  
  L = len(beta)
  
  b = torch.zeros(L)
  for l in range(L): 
    if l == 0 : 
      b[l] = beta[l]
    else: 
      b[l] = beta[l]-beta[l-1]
  d = torch.zeros(L)
  for l in range(1:L): 
    d[l] = torch.sum(delta[:l])
    
  return b, d

In [0]:
def crps_loss(theta, z):
  gamma, beta, delta = theta
  
  L = len(beta)
  
  b, d = getbd_from_theta(theta)
    
  for l in range(L, 0, -1): 
    val = sqf(theta, d[l])
    if val < z:
      lo = l
      break 
  
  a_tilde = (z-gamma + torch.sum(b[:lo]*d[:lo]))/torch.sum(b[:lo])
  
  max_ = torch.max(torch.zeros(L)+a_tilde, d)
  
  bracket = (1/3)*(1-torch.pow(d, 3)) - d - torch.pow(max_,2) + 2*max_*d
  
  loss = (2*a_tilde - 1)*z + (1-2*a_tilde)*gamma + torch.sum(b*bracket)
  
  return loss 

In [0]:
def sqf(theta, quantile): 
  
  #would expect beta and delta to be vectors length hidden_units
  
  gamma, beta, delta = theta
  L = len(beta)
  
  b,d = getbd_from_theta(theta)
  
  max_ = torch.max(q-d, torch.zeros(L))
  
  qf = gamma + torch.sum(b*max_)
  
  return qf

In [0]:
class Encoder(nn.module): 
  
  def __init__(self, input_size, hidden_size, batch_size, output_size, num_layers):
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.output_size = output_size
    self.num_layers = num_layers
    
    self.lstm = nn.LSTM(input_size = self.input_size, hidden_size = self.hidden_size, num_layers = self.num_layers)
    self.linear = nn.Linear(self.hidden_size, self.hidden_size)
    self.softmax = nn.functional.softmax
    self.softplus = nn.functional.softplus
    
  def init_hidden(self): 
    return (torch.zeros(self.num_layers, self.batch_size, self.hidden_size),
              torch.zeros(self.num_layers, self.batch_size, self.hidden_size))
  
  def forward(self, data, hidden):
    lstm_out, hidden = self.lstm(data.view(len(data), self.batch_size, -1))
    fc_layer = self.Linear()
    delta = self.softmax(fc_layer)
    beta = self.softmax(fc_layer)
    
    theta = (delta, beta)
    
    
    return theta, hidden #want the full sequence of hidden states as the output of the encoder layer NO we only want the last one?!
  
  ##does output of hidden give the outputs from every layer?

In [0]:
class Decoder(nn.module): 
  
  # want the decoder to share the same weights as the encoder - we won't train this?
  # but we need the decoder to take the output at the previous step to feed in as the next input 
  

In [0]:
### TRAINING the encoder###
encoder = Encoder(...)
loss_function = crps_loss()
encoder_optimiser = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
T = #decide this

for i in range(num_epochs): 
  encoder.zero_grad()
  encoder.hidden = encoder.init_hidden()
  loss_t = 0 
  
  for t in range(T): 
    theta, encoder_hidden = encoder(input_data[t], encoder_hidden)
    loss_t += loss_function(theta, input_data[t+1])
    
  loss.backward()

  encoder_optimizer.step()
    
   
    
    

In [0]:
learning_rate = 
learning_rate_decay = 